In [12]:
import wave
import struct
import numpy as np
import pyaudio
import librosa
import soundfile as sf
import scipy.io.wavfile as wf
import numpy as np
import base64
import hashlib
import json 
import requests
import hmac
import json
from datetime import datetime
from time import mktime
from urllib.parse import urlencode
from wsgiref.handlers import format_date_time
from pydub import AudioSegment

def SNR(origanl, current):
    noise=current-origanl
    # origanl_ar = sess.run(origanl)
    # noise_ar = sess.run(noise)
    ans = 20*np.log10(np.linalg.norm(origanl, ord=2)/np.linalg.norm(noise, ord=2))
    print("SNR:", ans)
def trans_wav_mp3(file_path):
    """
    将wav文件转为mp3文件，保存到cmu_mp3
    """
    sourcefile = AudioSegment.from_wav(file_path)
    #filename = file_path.split('/')[-1].split('.wav')[0].replace(' ', '_') + '.mp3'
    filename = file_path.split('/')[-1].split('.wav')[0] + '.mp3'
    save_path = "/mnt/data/Chenpinji/Two-Step-Attack-on-SRS/mp3/"
    #print(filename)
    sourcefile.export(save_path + filename, format="mp3")


x, _ = librosa.load('/mnt/data/Chenpinji/cmu_dataset/cmu3/wav/arctic_a0008.wav', sr=16000)  # 输入文件1：音频1(调节音量）sr：采样率（可以改）

volume = 0.5  # 调节x的音量（倍数）
x = x * volume
sf.write('t1.wav', x, 16000)

y, _ = librosa.load('/mnt/data/Chenpinji/cmu_dataset/cmu7/wav/arctic_a0008.wav', sr=16000)  # 输入文件2：音频2
sf.write('t2.wav', y, 16000)
f1 = wave.open('t1.wav', 'rb')
f2 = wave.open('t2.wav', 'rb')

# 音频1的数据
params1 = f1.getparams()
nchannels1, sampwidth1, framerate1, nframes1, comptype1, compname1 = params1[:6]
print(nchannels1, sampwidth1, framerate1, nframes1, comptype1, compname1)
f1_str_data = f1.readframes(nframes1)
f1.close()
f1_wave_data = np.frombuffer(f1_str_data, dtype=np.int16)

# 音频2的数据
params2 = f2.getparams()
nchannels2, sampwidth2, framerate2, nframes2, comptype2, compname2 = params2[:6]
print(nchannels2, sampwidth2, framerate2, nframes2, comptype2, compname2)
f2_str_data = f2.readframes(nframes2)
f2.close()
f2_wave_data = np.frombuffer(f2_str_data, dtype=np.int16)

# 对不同长度的音频用数据零对齐补位
if nframes1 < nframes2:
    length = abs(nframes2 - nframes1)
    temp_array = np.zeros(length, dtype=np.int16)
    rf1_wave_data = np.concatenate((f1_wave_data, temp_array))
    rf2_wave_data = f2_wave_data
elif nframes1 > nframes2:
    length = abs(nframes2 - nframes1)
    temp_array = np.zeros(length, dtype=np.int16)
    rf2_wave_data = np.concatenate((f2_wave_data, temp_array))
    rf1_wave_data = f1_wave_data
else:
    rf1_wave_data = f1_wave_data
    rf2_wave_data = f2_wave_data

# 合并1和2的数据
new_wave_data = rf1_wave_data + rf2_wave_data
new_wave_data = new_wave_data * 1.0 / (max(abs(new_wave_data)))  # wave幅值归一化
SNR(rf2_wave_data, new_wave_data)
new_wave = new_wave_data.tostring()

p = pyaudio.PyAudio()
CHANNELS = 1
FORMAT = pyaudio.paInt16

# 写文件
time = 10

# 打开WAV文档
f = wave.open(r"attackcmu3.wav", "wb")  # 文件3：输出音频

# 配置声道数、量化位数和取样频率
nchannels = 1  # 通道数 目前设置为1
sampwidth = 2
data_size = len(new_wave_data)
framerate = 16000  # 需要和前面的采样率保持一致
nframes = data_size
comptype = "NONE"
compname = "not compressed"
f.setparams((nchannels, sampwidth, framerate, nframes, comptype, compname))
# 将wav_data转换为二进制数据写入文件
# f.writeframes(new_wave)
for v in new_wave_data:
    f.writeframes(struct.pack('h', int(v * 64000 / 2)))
f.close()
trans_wav_mp3("attackcmu3.wav")
filepath = "/mnt/data/Chenpinji/Two-Step-Attack-on-SRS/mp3/attackcmu3.mp3"    #此条注释打开是一些cmu数据
value = req_score(api_name='searchFea', APPId=APPId,APIKey=APIKey, APISecret=APISecret, file_path=filepath)
print(value)
speaker1,score1,speaker2,score2 = GetSpeakerScore(value)#得到第一名和第二名得分
print('classified as:',speaker1,'Score is:',score1)
print('second place is:',speaker2, 'Score is:',score2)

1 2 16000 36560 NONE not compressed
1 2 16000 34000 NONE not compressed
SNR: 0.0004212094002331042


/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:85: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


{'header': {'code': 0, 'message': 'success', 'sid': 'ase000f25f2@hu18714438b671323882'}, 'payload': {'searchFeaRes': {'compress': 'raw', 'encoding': 'utf8', 'format': 'json', 'status': '3', 'text': 'eyJzY29yZUxpc3QiOlt7ImZlYXR1cmVJZCI6ImNtdV9jYW5hZGlhbiIsImZlYXR1cmVJbmZvIjoiYXJjdGljX2EwMDA4LndhdiIsInNjb3JlIjowLjQ3fSx7ImZlYXR1cmVJZCI6ImNtdTMiLCJmZWF0dXJlSW5mbyI6ImFyY3RpY19hMDAwOC53YXYiLCJzY29yZSI6MC40Mn0seyJmZWF0dXJlSWQiOiJjbXVfaW5kaWFuIiwiZmVhdHVyZUluZm8iOiJhcmN0aWNfYTAwMDgud2F2Iiwic2NvcmUiOjAuNH0seyJmZWF0dXJlSWQiOiJjbXU3IiwiZmVhdHVyZUluZm8iOiJhcmN0aWNfYTAwMDgud2F2Iiwic2NvcmUiOjAuMzh9LHsiZmVhdHVyZUlkIjoiY211MSIsImZlYXR1cmVJbmZvIjoiYXJjdGljX2EwMDA4LndhdiIsInNjb3JlIjowLjI2fSx7ImZlYXR1cmVJZCI6ImNtdTYiLCJmZWF0dXJlSW5mbyI6ImFyY3RpY19hMDAwOC53YXYiLCJzY29yZSI6MC4yNX0seyJmZWF0dXJlSWQiOiJjbXU0IiwiZmVhdHVyZUluZm8iOiJhcmN0aWNfYTAwMDgud2F2Iiwic2NvcmUiOjAuMjR9LHsiZmVhdHVyZUlkIjoiY211MiIsImZlYXR1cmVJbmZvIjoiYXJjdGljX2EwMDA4LndhdiIsInNjb3JlIjowLjE5fSx7ImZlYXR1cmVJZCI6ImNtdV9zY290dGlzaCIsImZlYXR1cmVJb

In [7]:
#wzx账号 组号1是cmu
APPId = "e982a7e0"
APISecret = "N2JhMzM2Yzg2ZmZhMDUyNTRkZDZlYjA3"
APIKey = "b9aeab32dc7169a17e54e65a1b8dfddc"
class Gen_req_url(object):
    def sha256base64(self, data):
        sha256 = hashlib.sha256()
        sha256.update(data)
        digest = base64.b64encode(sha256.digest()).decode(encoding='utf-8')
        return digest

    def parse_url(self, requset_url):
        stidx = requset_url.index("://")
        host = requset_url[stidx + 3:]
        # self.schema = requset_url[:stidx + 3]
        edidx = host.index("/")
        if edidx <= 0:
            raise Exception("invalid request url:" + requset_url)
        self.path = host[edidx:]
        self.host = host[:edidx]

    # build websocket auth request url
    def assemble_ws_auth_url(self, requset_url, api_key, api_secret, method="GET"):
        self.parse_url(requset_url)
        now = datetime.now()
        date = format_date_time(mktime(now.timetuple()))
        # date = "Thu, 12 Dec 2019 01:57:27 GMT"
        signature_origin = "host: {}\ndate: {}\n{} {} HTTP/1.1".format(self.host, date, method, self.path)
        signature_sha = hmac.new(api_secret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha256).digest()
        signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')
        authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
            api_key, "hmac-sha256", "host date request-line", signature_sha)
        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
        values = {
            "host": self.host,
            "date": date,
            "authorization": authorization
        }

        return requset_url + "?" + urlencode(values)


def gen_req_body(apiname, APPId, file_path=None):
    if apiname == 'createFeature':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "createFeature",
                    "groupId": "1",
                    "featureId": "cmu_scottish",
                    "featureInfo": "wav08",
                    "createFeatureRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    elif apiname == 'createGroup':

        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "createGroup",
                    "groupId": "8",
                    "groupName": "cmu_dataset",
                    "groupInfo": "cmu_dataset",
                    "createGroupRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            }
        }
   
    elif apiname == 'queryFeatureList':

        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "queryFeatureList",
                    "groupId": "8",
                    "queryFeatureListRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            }
        }
    elif apiname == 'searchFea':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "searchFea",
                    "groupId": "1",#1是cmu10   2是Voxceleb1 10
                    "topK": 10,
                    "searchFeaRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    elif apiname == 'searchScoreFea':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "searchScoreFea",
                    "groupId": "iFLYTEK_examples_groupId",
                    "dstFeatureId": "p374",
                    "searchScoreFeaRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    else:
        raise Exception("输入的apiname不在[createFeature, createGroup, deleteFeature, queryFeatureList, searchFea, searchScoreFea,updateFeature]内，请检查")
    return body

def req_url(api_name, APPId, APIKey, APISecret, file_path=None):
    """
    开始请求
    :param APPId: APPID
    :param APIKey:  APIKEY
    :param APISecret: APISecret
    :param file_path: body里的文件路径
    :return:
    """
    gen_req_url = Gen_req_url()
    body = gen_req_body(apiname=api_name, APPId=APPId, file_path=file_path)
    request_url = gen_req_url.assemble_ws_auth_url(requset_url='https://api.xf-yun.com/v1/private/s782b4996', method="POST", api_key=APIKey, api_secret=APISecret)
    headers = {'content-type': "application/json", 'host': 'api.xf-yun.com', 'appid': '$APPID'}
    response = requests.post(request_url, data=json.dumps(body), headers=headers)
    tempResult = json.loads(response.content.decode('utf-8'))
    #print(tempResult)
    #print(base64.b64decode(tempResult['payload'][api_name + 'Res']['text']))
"""
 * 1.声纹识别接口,请填写在讯飞开放平台-控制台-对应能力页面获取的APPID、APIKey、APISecret
 * 2.groupId要先创建,然后再在createFeature里使用,不然会报错23005,修改时需要注意保持统一
 * 3.音频base64编码后数据(不超过4M),音频格式需要16K、16BIT的MP3音频。
 * 4.主函数只提供调用示例,其他参数请到对应类去更改,以适应实际的应用场景。
"""
def req_score(api_name, APPId,  APIKey, APISecret, file_path=None):
    gen_req_url = Gen_req_url()
    body = gen_req_body(apiname=api_name, APPId=APPId, file_path=file_path)
    request_url = gen_req_url.assemble_ws_auth_url(requset_url='https://api.xf-yun.com/v1/private/s782b4996', method="POST", api_key=APIKey, api_secret=APISecret)
    headers = {'content-type': "application/json", 'host': 'api.xf-yun.com', 'appid': '$APPID'}
    response = requests.post(request_url, data=json.dumps(body), headers=headers)
    tempResult = json.loads(response.content.decode('utf-8'))
    print(tempResult)
    return base64.b64decode(tempResult['payload'][api_name + 'Res']['text'])#,base64.b64decode(tempResult['payload'][api_name + 'Res']['text']['scoreList'][0]['score'])


def GetSpeakerScore(value):
    m = len(value)
    result = ''
    for i in range(0,m):
        result+=chr(value[i]) #switch to string
    result = eval(result)
    v = list(result.values())[0]
    #print(v)
    lenv=len(v)
    return v[0]['featureId'], v[0]['score'], v[1]['featureId'],v[1]['score']